### Jimmy Wrangler, Data Explorer : Traveling the world on a mission to discover new data

#### The objectives of this project are:
<ol>
<li> Find public data sets </li>
<li> Do exploratory data analysis by combining them </li>
</ol>

Project Idea:
<b>homelessness count vs Total resident population</b>

Please refer readme for detailed report.

Datasets:
<ol>
<li> Source - 1 : <a href="https://www.hudexchange.info/resources/documents/2007-2018-PIT-Counts-by-CoC.xlsx"> Homelessness Data </a> </li>
<li> Source - 2 : <a href="https://www2.census.gov/programs-surveys/popest/tables/2010-2018/state/totals/nst-est2018-01.xlsx"> United States Census Estimate Data</a> </li>
</ol>

In [1]:
# Imports

In [2]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
# reading datasets

In [4]:
df = pd.read_excel("../data/external/2007-2018-PIT-Counts-by-CoC.xlsx", sheet_name=None)
df2 = pd.read_excel("../data/external/nst-est2018-01.xlsx", sheet_name=None)

In [5]:
print(df.keys())

odict_keys(['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', 'CoC Mergers', 'Revisions'])


In [6]:
# Select only 2018 homelessness data

In [7]:
homelessness = df["2018"]

In [8]:
homelessness.head()

,CoC Number,CoC Name,CoC Category,"Overall Homeless, 2018","Sheltered ES Homeless, 2018","Sheltered TH Homeless, 2018","Sheltered SH Homeless, 2018","Sheltered Total Homeless, 2018","Unsheltered Homeless, 2018","Homeless Individuals, 2018",...,"Homeless Parenting Youth Age 18-24, 2018","Sheltered ES Homeless Parenting Youth Age 18-24, 2018","Sheltered TH Homeless Parenting Youth Age 18-24, 2018","Sheltered Total Homeless Parenting Youth Age 18-24, 2018","Unsheltered Homeless Parenting Youth Age 18-24, 2018","Homeless Children of Parenting Youth, 2018","Sheltered ES Homeless Children of Parenting Youth, 2018","Sheltered TH Homeless Children of Parenting Youth, 2018","Sheltered Total Homeless Children of Parenting Youth, 2018","Unsheltered Homeless Children of Parenting Youth, 2018"
0,AK-500,Anchorage CoC,Other Urban CoCs,1094,751,249,0,1000,94,814,...,14,6,8,14,0,14,5,9,14,0
1,AK-501,Alaska Balance of State CoC,Rural CoCs,922,497,210,0,707,215,620,...,11,8,1,9,2,12,9,1,10,2
2,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",Suburban CoCs,901,431,219,32,682,219,720,...,3,1,2,3,0,7,1,6,7,0
3,AL-501,Mobile City & County/Baldwin County CoC,Other Urban CoCs,551,225,93,0,318,233,399,...,9,4,1,5,4,12,8,1,9,3
4,AL-502,Florence/Northwest Alabama CoC,Rural CoCs,256,128,106,0,234,22,166,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Remove unwanted columns and rows - basically I only kept CoC number and overall Homelessness and removed
# the last 3 rows which are unwanted

In [11]:
cols = [x for x in range(len(homelessness.columns))][4:] + [1,2]

homelessness.drop(homelessness.columns[cols], axis=1, inplace=True)
homelessness.drop(homelessness.index[[-1,-2,-3]], axis=0, inplace=True)

In [12]:
homelessness.head()

,CoC Number,"Overall Homeless, 2018"
0,AK-500,1094
1,AK-501,922
2,AL-500,901
3,AL-501,551
4,AL-502,256


In [14]:
homelessness.tail()

,CoC Number,"Overall Homeless, 2018"
393,WV-500,108
394,WV-501,190
395,WV-503,317
396,WV-508,628
397,WY-500,639


In [15]:
# There is no "state" in this dataset, but state can be predicted from CoC number
# The dictionary of country codes and state name will be a good way
# I quickly made one!

In [16]:
state_code = {'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AZ': 'Arizona', 'CA': 'California',
        'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida',
        'GA': 'Georgia', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois',
        'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland',
        'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota', 'MO': 'Missouri',
        'MS': 'Mississippi', 'MT': 'Montana', 'NC': 'North Carolina', 'ND': 'North Dakota',
        'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NV': 'Nevada',
        'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
        'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee',
        'TX': 'Texas', 'UT': 'Utah', 'VA': 'Virginia', 'VT': 'Vermont', 'WA': 'Washington', 'WI': 'Wisconsin',
        'WV': 'West Virginia', 'WY': 'Wyoming'}

In [19]:
# Now separating CoC numbers to get state codes
# Adding "state" as new column by using state_code dictionary and dropping "CoC value" column
# Also converting overall homeless count to int
# As can be seen in the above dictionary, spacial US territories are excluded
# Also we still need state codes to plot geolocation graph

In [20]:
for i in homelessness.index.values.tolist():
    if homelessness.at[i, "Overall Homeless, 2018"] == ".":
        continue
    try:
        homelessness.at[i, "State"] = state_code[str(homelessness.at[i, "CoC Number"])[:2]]
        # We also need the state code
        homelessness.at[i, "State_code"] = str(homelessness.at[i, "CoC Number"])[:2]
    except:
        continue
    homelessness.at[i, "Overall Homeless, 2018"] = int(homelessness.at[i, "Overall Homeless, 2018"])

homelessness.drop(homelessness.columns[0], axis=1, inplace=True)

In [22]:
homelessness.head()

,"Overall Homeless, 2018",State,State_code
0,1094,Alaska,AK
1,922,Alaska,AK
2,901,Alabama,AL
3,551,Alabama,AL
4,256,Alabama,AL


In [23]:
# Using groupby function to sum number of homeless of the same state
homelessness = homelessness.groupby(['State', 'State_code'])['Overall Homeless, 2018'].sum().reset_index()

In [25]:
homelessness.tail()

,State,State_code,"Overall Homeless, 2018"
46,Virginia,VA,5975
47,Washington,WA,22304
48,West Virginia,WV,1243
49,Wisconsin,WI,4907
50,Wyoming,WY,639


In [26]:
# Selecting census data

In [27]:
df2.keys()

odict_keys(['NST01'])

In [28]:
census = df2["NST01"]

In [29]:
# Removing unwanted rows and columns (i.e. selecting the data only for 2018)
# Also setting convenient column names

In [30]:
# Removing unwanted rows and columns
census.drop(census.columns[1:11], axis=1, inplace=True)
census.drop(census.index[[0,1,2,3,4,5,6,7]], axis=0, inplace=True)
census.drop(census.index[[-7,-6,-5,-4,-3,-2,-1]], axis=0, inplace=True)

In [31]:
census.columns = ["State", "Population (2018)"]

In [32]:
# Cleaning '.'s from state names
for i in census.index.values.tolist():
    census.at[i, "State"] = str(census.at[i, "State"]).replace(".", "")

In [33]:
census.head()

,State,Population (2018)
8,Alabama,4887871.0
9,Alaska,737438.0
10,Arizona,7171646.0
11,Arkansas,3013825.0
12,California,39557045.0


In [34]:
census = census.reset_index(drop=True)

In [36]:
census.head()

,State,Population (2018)
0,Alabama,4887871.0
1,Alaska,737438.0
2,Arizona,7171646.0
3,Arkansas,3013825.0
4,California,39557045.0


In [37]:
homelessness.tail()

,State,State_code,"Overall Homeless, 2018"
46,Virginia,VA,5975
47,Washington,WA,22304
48,West Virginia,WV,1243
49,Wisconsin,WI,4907
50,Wyoming,WY,639


In [38]:
census.tail()

,State,Population (2018)
46,Virginia,8517685.0
47,Washington,7535591.0
48,West Virginia,1805832.0
49,Wisconsin,5813568.0
50,Wyoming,577737.0


In [39]:
merged = homelessness.merge(census, on="State")

In [40]:
merged.head()

,State,State_code,"Overall Homeless, 2018",Population (2018)
0,Alabama,AL,3434,4887871.0
1,Alaska,AK,2016,737438.0
2,Arizona,AZ,9865,7171646.0
3,Arkansas,AR,2712,3013825.0
4,California,CA,129972,39557045.0


In [41]:
# The dataframes have been combined.
# Now plotting homelessness percentage

In [42]:
# let's zoom in a little
# Calulating total homeless percentage

Y = [(i/j)*100 for i,j in zip(list(merged["Overall Homeless, 2018"]), list(merged["Population (2018)"]))]

data = [go.Bar(x=list(merged["State"]),
               y=Y)]

layout = go.Layout(barmode='stack', title = 'Estimated Percentage of Total Homeless population by State (2018)')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [43]:
# Plotting the same data with per 10000 population

Y = [(i/j)*10000 for i,j in zip(list(merged["Overall Homeless, 2018"]), list(merged["Population (2018)"]))]

data = [go.Bar(x=list(merged["State"]),
               y=Y, marker={"color":"lightslategrey"})]

layout = go.Layout(barmode='stack')

fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title=go.layout.Title(
        text="Estimated Homeless Population Count by State (2018) - (Per 10000 Total Population)",
        xref="paper",
        x=0,
        font=dict(
                size=14,
            )
    ))
iplot(fig)

In [44]:
# Now plotting the geolocation graph
# But then state code is needed which we already have
# Ref: https://plot.ly/python/choropleth-maps/

Y = [(i/j)*100 for i,j in zip(list(merged["Overall Homeless, 2018"]), list(merged["Population (2018)"]))]

fig = go.Figure(data=go.Choropleth(
    locations=merged['State_code'], # Spatial coordinates
    z = Y, # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "% homelessness",
))

fig.update_layout(
    title_text = 'Estimated Percentage of Total Homeless population by State (2018)',
    geo_scope='usa', # limite map scope to USA
)

fig.show()